# Aesthetics Score Training

The task is to train a Deep Learning model to predict the score given an image based on the Aesthetics of the image. We will use the Standard AVA dataset for the training.

In [ ]:
# Imports
from os import path
import json
from random import sample

import numpy as np
import cv2

#%matplotlib inline
#import matplotlib.pylab as plt

In [ ]:
def get_score(scores, labels, N_class):
    y = np.zeros((N_class, ))
    for idx, score in enumerate(scores):
        map_idx = int(labels[str(idx + 1)]) - 1
        y[map_idx] += score
    y = (y / sum(y)) if sum(y) else y
    return y

In [ ]:
def load_imgs_and_label(data_file, labels_file, n_imgs):
    with open(labels_file, 'r') as fi:
        labels = json.load(fi)
    with open(data_file, 'r') as fi:
        data = fi.readlines()

    # Initialize matrices
    data = sample(data, n_imgs)
    count = 0
    N_class = len(set(labels.values()))
    total_imgs = len(data)
    print(total_imgs)
    X = np.empty((total_imgs, 224, 224, 3))
    Y = np.empty((total_imgs, N_class))

    for line in data:
        values = line.rstrip().split()
        scores = [int(v) for v in values[2: 12]]
        y_score = get_score(scores, labels, N_class)

        img_name = values[1] + '.jpg'
        img_path = path.join(all_imgs_dir, img_name[:2], img_name)
        img = cv2.imread(img_path)
        try:
            # change img scale
            img = (img - 127.0) / 127.0

            if img.shape[2] == 4:  # PNG image
                img = img[:, :, :-1]
            elif img.shape[2] == 1:  # Gray image
                cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
            img = img.reshape(1, img.shape[0], img.shape[1], 3)
        except:
            continue
            print('************IMG reshaping issue******')

        y_score = y_score.reshape(1, N_class)

        X[count] = img
        Y[count] = y_score
        count += 1
        if not (count % 100):
            print('Processed ==>', count)
    X = X[:count]
    Y = Y[:count]
    return X, Y

In [ ]:
def load_data_from_imgs(dataset_dir):
    dataset_name = path.split(dataset_dir)[1]
    train_file = path.join(dataset_dir, dataset_name + '_train.txt')
    val_file = path.join(dataset_dir, dataset_name + '_train.txt')
    test_file = path.join(dataset_dir, dataset_name + '_test.txt')
    labels_file = path.join(dataset_dir, dataset_name + '_labels.json')

    train_x, train_y = load_imgs_and_label(train_file, labels_file, 1000)
    val_x, val_y = load_imgs_and_label(val_file, labels_file, 200)
    data = {'train': (train_x, train_y), 'val': (val_x, val_y)}
    if path.exists(test_file):
        test_x, test_y = load_imgs_and_label(test_file, labels_file, 200)
        data['test'] = (test_x, test_y)
    return data

In [ ]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, Conv2D
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import SGD, Adam, Adadelta

In [ ]:
# Define basic CPCPDHDH cnn architecture
def model_CPCPDHDH(IMAGE_SIZE, N_class):
    nb_filter_1 = 20
    nb_filter_2 = 50
    nb_conv_1 = 5
    nb_conv_2 = 3

    cnn = Sequential()
    cnn.add(Convolution2D(nb_filter_1, nb_conv_1, activation='relu',
                          input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)))
    cnn.add(MaxPooling2D(strides=(2, 2)))

    cnn.add(Convolution2D(nb_filter_2, nb_conv_2, activation='relu'))
    cnn.add(MaxPooling2D(strides=(2, 2)))

    cnn.add(Flatten())
    cnn.add(Dropout(0.3))
    cnn.add(Dense(500, activation='relu'))

    cnn.add(Dropout(0.3))
    cnn.add(Dense(500, activation='relu'))

    cnn.add(Dense(N_class, activation="softmax"))

    adadelta = Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0)

    cnn.compile(loss='categorical_crossentropy',
                optimizer='adadelta',
                metrics=['accuracy'])
    cnn.summary()
    return cnn

In [ ]:
# Configurations for the training
N_class = 2
batch_size = 50
epochs = 5
image_size = 224

# More advanced configurations
learning_rate = 0.01
optimizer = 'adadelta'
val_split = 0.2

# Data directories
all_imgs_dir = '/home/dhivakar/work/projects/scancafe/sl_git/sc_as/data/ava/subsets/ahws_18/ava_img_groups'
dataset_dir = '/home/dhivakar/work/projects/scancafe/sl_git/sc_as/data/ava/subsets/ahws_18/datasets/ava_5k_2cls'
model_name = 'ava_5k_2cls.h5'
model_path = path.join(dataset_dir, model_name)

In [ ]:
expt_data = load_data_from_imgs(dataset_dir)

In [ ]:
X_train, Y_train = expt_data['train']
X_val, Y_val = expt_data['val']
print(X_train.shape, Y_train.shape)
print(X_val.shape, Y_val.shape)

In [ ]:
model = model_CPCPDHDH(image_size, N_class)

In [ ]:
history = model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=epochs, validation_split=val_split)

In [ ]:
model.save(model_loc)
print('Training accuracy: ', history.history['acc'][-1])
print('Validation accuracy: ', history.history['val_acc'][-1])

In [ ]:
%matplotlib inline  
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.legend(['training','validation'], loc = 'upper left')
plt.show()

# 10 Class

As we verified the base case, now we will move on to the original problem of rating a photo from 1 to 10

In [ ]:
# Configurations for the training
N_class = 2
batch_size = 50
epochs = 5
image_size = 224

# More advanced configurations
learning_rate = 0.01
optimizer = 'adadelta'
val_split = 0.2

# Data directories
all_imgs_dir = '/home/dhivakar/work/projects/scancafe/sl_git/sc_as/data/ava/subsets/ahws_18/ava_img_groups'
dataset_dir = '/home/dhivakar/work/projects/scancafe/sl_git/sc_as/data/ava/subsets/ahws_18/datasets/ava_10cls_10k'
model_name = 'ava_10cls_10k.h5'
model_path = path.join(dataset_dir, model_name)

In [ ]:
expt_data = load_data_from_imgs(dataset_dir)

In [ ]:
X_train, Y_train = expt_data['train']
X_val, Y_val = expt_data['val']
print(X_train.shape, Y_train.shape)
print(X_val.shape, Y_val.shape)

In [ ]:
model = model_CPCPDHDH(image_size, N_class)

In [ ]:
history = model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=epochs, validation_split=val_split)

In [ ]:
model.save(model_loc)
print('Training accuracy: ', history.history['acc'][-1])
print('Validation accuracy: ', history.history['val_acc'][-1])

In [ ]:
%matplotlib inline  
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.legend(['training','validation'], loc = 'upper left')
plt.show()

# Class Imbalance Problem

If the training data is not distributed uniformly among all the classes, for example, a particular class has more data than the other classes, the model tends to be biased towards that particular class. We call this a Class Imbalance.

In [ ]:
# Imports
from keras.models import load_model

In [ ]:
# Load model
loaded_model = load_model(model_loc)

In [ ]:
X_test, Y_test = expt_data['test']
print(X_test.shape, Y_test.shape)

In [ ]:
pred_arr = loaded_model.predict(X_test, verbose=1)
y_labels = np.argmax(Y_test)
pred_labels = np.argmax(pred_arr)
correct_preds = np.sum(y_labels == pred_labels)
accuracy = correct_preds / len(y_labels) * 100
print(accuracy)

Display the data distribution and understand how to improve

In [ ]:
y_train_labels = np.argmax(Y_train)
y_categ_count = np.unique(y_train_labels, return_counts=True)
plt_labels, plt_height = list(zip(*sorted(zip(*np.unique(m, return_counts=True)))))

In [ ]:
plt.bar(plt_labels, plt_height)

# Data Sampling

A straight forward idea to solve this problem is to balance the Training set. Choose the minimum possible count and choose those many images from all classes

As we don't have enough images, we bucket them into 5 classes each with 1k images

In [ ]:
# Configurations for the training
N_class = 5
batch_size = 50
epochs = 5
image_size = 224

# More advanced configurations
learning_rate = 0.01
optimizer = 'adadelta'
val_split = 0.2

# Data directories
all_imgs_dir = '/home/dhivakar/work/projects/scancafe/sl_git/sc_as/data/ava/subsets/ahws_18/ava_img_groups'
dataset_dir = '/home/dhivakar/work/projects/scancafe/sl_git/sc_as/data/ava/subsets/ahws_18/datasets/ava_1.5k_5cls'
model_name = 'ava_1.5k_5cls.h5'
model_path = path.join(dataset_dir, model_name)

In [ ]:
expt_data = load_data_from_imgs(dataset_dir)

In [ ]:
X_train, Y_train = expt_data['train']
X_val, Y_val = expt_data['val']
print(X_train.shape, Y_train.shape)
print(X_val.shape, Y_val.shape)

In [ ]:
model = model_CPCPDHDH(image_size, N_class)

In [ ]:
history = model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=epochs, validation_split=val_split)

In [ ]:
model.save(model_loc)
print('Training accuracy: ', history.history['acc'][-1])
print('Validation accuracy: ', history.history['val_acc'][-1])

In [ ]:
%matplotlib inline  
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.legend(['training','validation'], loc = 'upper left')
plt.show()